## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-08-13-00-53 +0000,wapo,Trump’s crackdown on foreign students barely i...,https://www.washingtonpost.com/education/2025/...
1,2025-11-08-12-55-58 +0000,nypost,NYPD already losing officers with morale plung...,https://nypost.com/2025/11/08/us-news/nypd-alr...
2,2025-11-08-12-45-55 +0000,nyt,What Travelers Need to Know About the Shutdown...,https://www.nytimes.com/article/shutdown-faa-f...
3,2025-11-08-12-38-25 +0000,nyt,Pfizer to Acquire Obesity Drug Start-Up Metser...,https://www.nytimes.com/2025/11/08/business/me...
4,2025-11-08-12-02-46 +0000,bbc,Council launches winter energy cost support sc...,https://www.bbc.com/news/articles/c8x1y5p8epvo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,47
32,up,14
23,shutdown,13
24,flight,11
97,court,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
98,2025-11-08-02-02-29 +0000,nypost,Supreme Court allows Trump admin to temporaril...,https://nypost.com/2025/11/07/us-news/trump-ad...,112
100,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...,104
88,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...,102
90,2025-11-08-02-59-00 +0000,wsj,Supreme Court Pauses Order for Trump Administr...,https://www.wsj.com/us-news/law/supreme-court-...,101
150,2025-11-07-22-23-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
98,112,2025-11-08-02-02-29 +0000,nypost,Supreme Court allows Trump admin to temporaril...,https://nypost.com/2025/11/07/us-news/trump-ad...
100,57,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...
157,52,2025-11-07-22-06-00 +0000,wsj,Defense Secretary Pete Hegseth said that the P...,https://www.wsj.com/politics/national-security...
5,46,2025-11-08-12-00-00 +0000,nypost,Zohran Mamdani’s wife Rama Duwaji uses her art...,https://nypost.com/2025/11/08/us-news/zohran-m...
8,41,2025-11-08-11-30-07 +0000,wapo,"UPS, FedEx ground MD-11 planes after Louisvill...",https://www.washingtonpost.com/nation/2025/11/...
149,39,2025-11-07-22-31-55 +0000,bbc,Tesla plans to pay Musk $1tn - do they really ...,https://www.bbc.com/news/articles/c5yp8w5k5q9o...
295,39,2025-11-07-14-11-00 +0000,wsj,Senate Majority Leader John Thune expects Sena...,https://www.wsj.com/politics/policy/government...
267,36,2025-11-07-16-04-00 +0000,wsj,Heritage Foundation president Kevin Roberts de...,https://www.wsj.com/politics/policy/the-crack-...
120,35,2025-11-08-00-06-04 +0000,nypost,Michigan woman captured hurling piping hot cof...,https://nypost.com/2025/11/07/us-news/michigan...
130,34,2025-11-07-23-36-31 +0000,wapo,Flight cancellations hit major airports on fir...,https://www.washingtonpost.com/transportation/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
